In [12]:
import cv2
import numpy as np
import pytesseract
import imutils
import warnings
from helper import conversions
warnings.filterwarnings(action='ignore')

In [2]:
def preprocess_input(img,contrast = True):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    filtered = cv2.adaptiveThreshold(img.astype(np.uint8), 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 41)
    kernel = np.ones((1, 1), np.uint8)
    opening = cv2.morphologyEx(filtered, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    or_image = cv2.bitwise_or(img, closing)
    #Adding contrast to the image for better clarity
    if contrast:
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        or_image = cv2.filter2D(or_image, -1, kernel)
    return or_image

In [3]:
def order_points(pts):
        rect = np.zeros((4, 2), dtype = "float32")
        s = pts.sum(axis = 1)
        rect[0] = pts[np.argmin(s)]
        rect[2] = pts[np.argmax(s)]
        diff = np.diff(pts, axis = 1)
        rect[1] = pts[np.argmin(diff)]
        rect[3] = pts[np.argmax(diff)]
        return rect

In [4]:
def four_point_transform(image, pts):
        rect = order_points(pts)
        (tl, tr, br, bl) = rect
        widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
        widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
        maxWidth = max(int(widthA), int(widthB))
        heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
        heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
        maxHeight = max(int(heightA), int(heightB))
        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]], dtype = "float32")
        M = cv2.getPerspectiveTransform(rect, dst) #This i just getting the matrix perspective for transformation 
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight)) #This is just a geometric transformation done on the image
        return warped

In [5]:
def get(img = None,vedio = False,path = 'Screenshot-6-12.png'):
    
    frame,image = None,None
    if vedio:
        frame,image = img,img
    else:
        frame = cv2.imread(path)    
        image = frame.copy()

    #if ROI:
       # x,y,w,h = cv2.selectROI(image)
       # image = image[x:x+w,y:y+h] 

    image = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    edged = cv2.Canny(image, 50, 100)
    cnts = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    contours = sorted(cnts , key = cv2.contourArea, reverse = True)
    contours = contours[0]

    #checking if the contour is a rectangle    
    screenCnt = None
    perimeter = cv2.arcLength(contours, True)
    approx = cv2.approxPolyDP(contours, 0.02 * perimeter, True)
    if len(approx) == 4:
                 screenCnt = approx

    #filtering on the basis of contour area        
    if cv2.contourArea(contours)>=1000:
            frame = cv2.drawContours(frame,[screenCnt],-1,[0,255,0],2)

    image = cv2.threshold(image,127,255,cv2.THRESH_BINARY)[1]
    image2 = preprocess_input(frame)

    image = cv2.erode(image,kernel = np.ones((2,2)),iterations = 2)
    image = cv2.dilate(image,kernel =  np.ones((2,2)),iterations = 2)
    image = cv2.GaussianBlur(image,(5,5),0)
    return frame,image2

In [15]:
def runApp(vedio = False,Text = False,Document = False):
    
    if vedio:
        cnt = 0
        cap  = cv2.VideoCapture(0)
        while True:
            ret,frame = cap.read()
            cv2.circle(frame,(200,300),2,[255,0,0],-1)
            cv2.circle(frame,(400,300),2,[255,0,0],-1)
            cv2.circle(frame,(400,50),2,[255,0,0],-1)
            cv2.circle(frame,(200,50),2,[255,0,0],-1)
            pts =  np.array([[200,300],[400,300],[400,50],[200,50]])
            wrapped = four_point_transform(frame,pts)
            filtered = get(wrapped,vedio = True)[1]
            cv2.imshow('F',frame)
            cv2.imshow('waapped',wrapped)
            cv2.imshow('filtered',filtered)
            
            if cv2.waitKey(1)& 0xFF == ord('s'): #take a screenshot if 's' is pressed
                cv2.imwrite('text'+str(cnt)+'.png',filtered) 
                cnt = cnt + 1
                break
            
            k = cv2.waitKey(30) & 0xff
            if k == 27:
                break

        cap.release()
        cv2.destroyAllWindows()
        
    else :
            frame = get()[1]
            text = pytesseract.image_to_string(frame)
            print(text)
            conv = conversion(text)
            conv.createPDF('new')
            cv2.imshow('frame',frame)
            cv2.imwrite('filtered.jpg',frame)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

In [16]:
runApp()

Berqhotel
Grosse Scheidegg
3818 Grindelvald
Feni lie R. Miler

 

Rech.Nr. 4572 30, 07, 2007/ 13:29: 17
Bar Tisch 7/01
exLatte Nacchiato a 4.50 CHF 9.00
IxGlokt a 5.00 CHF 5.00
IxSchweingchnitzel a 22.00 CHF 22.00
Inthasspatz 14 a 18.50 CHF 18,50

Total : CHF 54,50

Incl. 7.6% HwSt 54.50 CHF: 3.85

 

Entsoricht in Euro 36.33 EUR
Es bediente Sie: Ursula

HwSt Ne.: 930 254
Tel. + 033 853 87 16
Fax. + 033 853 67 19
E-wail: grossescheideggibluewin, ch
